In [ ]:
# --- 0. 安裝必要的函式庫 ---
!pip install --quiet pandas gspread plotly google-auth-oauthlib

# --- 1. 導入函式庫並進行授權 ---
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from google.colab import auth
import gspread
from google.auth import default
from IPython.display import HTML # 用於顯示最終的互動式 HTML 圖表

# 授權 Colab 存取你的 Google 帳號
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)
print("✅ Google Colab 授權成功。")

# --- 2. 讀取並清理 Google Sheet 資料 ---
sheet_url = "https://docs.google.com/spreadsheets/d/17d4pELhu5ylTPHsG3mmbSDu1Fh9rzDWugqFZYJ1XAg0"
gid = '1899526091' # 請確認這個 GID 對應到你新格式的工作表

# 建立 CSV 下載連結
csv_export_url = f'{sheet_url.split("/edit")[0]}/export?format=csv&gid={gid}'

try:
    df = pd.read_csv(csv_export_url)
    print("✅ 成功讀取 Google Sheet 資料。")

    # --- 3. 資料清理與準備 (升級版) ---
    # 重新命名所有需要的欄位
    df.rename(columns={
        '互動率（互動 / 觸及）': 'engagement_rate',
        '觸及人數': 'reach',
        '貼文 ID': 'post_id',
        '超連結': 'link',
        '貼文內容': 'content',
        '主題標籤': 'topic_tag',
        '類型標籤': 'type_tag'
    }, inplace=True)

    # 確保核心數據欄位存在
    required_cols = ['engagement_rate', 'reach', 'post_id', 'link', 'content', 'topic_tag']
    if not all(col in df.columns for col in required_cols):
        raise ValueError(f"工作表缺少必要欄位，請確認包含: {required_cols}")

    # --- 數據格式轉換 ---
    df['engagement_rate'] = pd.to_numeric(df['engagement_rate'].astype(str).str.replace('%', ''), errors='coerce') / 100
    df['reach'] = pd.to_numeric(df['reach'].astype(str).str.replace(',', ''), errors='coerce')
    df['topic_tag'].fillna('未分類', inplace=True)
    df['content'].fillna('', inplace=True)
    df.dropna(subset=['engagement_rate', 'reach'], inplace=True)

    # --- 建立用於圖表的衍生欄位 ---
    df['content_short'] = df['content'].str[:50] + '...'

    # 【修改 #1】改用中位數 (median) 作為基準
    median_reach = df['reach'].median()
    median_engagement_rate = df['engagement_rate'].median()

    # 定義判斷象限的函式 (現在根據中位數判斷)
    def get_quadrant(row):
        is_high_reach = row['reach'] >= median_reach
        is_high_er = row['engagement_rate'] >= median_engagement_rate
        if is_high_reach and is_high_er:
            return '王牌貼文'
        elif not is_high_reach and is_high_er:
            return '潛力珍寶'
        elif is_high_reach and not is_high_er:
            return '廣傳陷阱'
        else:
            return '常態內容'

    # 建立「象限」欄位
    df['quadrant'] = df.apply(get_quadrant, axis=1)

    print("\n--- 數據分析 (基準: 中位數) ---")
    print(f"中位數觸及人數: {median_reach:,.0f}")
    print(f"中位數互動率: {median_engagement_rate:.1%}")
    print("✅ 資料清理與準備完成。")


    # --- 4. 使用 Plotly 繪製圖表 (升級版) ---
    fig = px.scatter(
        df,
        x='reach',
        y='engagement_rate',
        color='topic_tag',
        title='非廣告貼文成效分析',
        custom_data=['link', 'content_short', 'quadrant', 'post_id']
    )


    fig.update_traces(
        hovertemplate="<br>".join([
            "<b>%{customdata[3]}</b>",
            "----------",
            "<b>內容預覽:</b> %{customdata[1]}",
            "<b>象限:</b> %{customdata[2]}",
            "<b>觸及人數:</b> %{x:,.0f}",
            "<b>互動率:</b> %{y:.1%}",
            "<extra></extra>"
        ])
    )

    # 加上代表中位數的中心線
    fig.add_vline(x=median_reach, line_dash="dash", line_color="grey")
    fig.add_hline(y=median_engagement_rate, line_dash="dash", line_color="grey")

    fig.update_layout(
        xaxis_title='觸及人數 (擴散力)',
        yaxis_title='互動率 (吸引力)',
        yaxis_tickformat='.1%',
        xaxis_tickformat=',.0f',
        title_x=0.5,
        height=700,
        legend_title_text='主題標籤'
    )

    # --- 5. 注入 JavaScript 實現點擊功能 ---
    plot_html = fig.to_html(full_html=False, include_plotlyjs='cdn', div_id='plotly-div')

    js_script = """
    <script>
        var plot_div = document.getElementById('plotly-div');
        plot_div.on('plotly_click', function(data){
            if(data.points.length > 0) {
                var point = data.points[0];
                var url = point.customdata[0];
                if(url) {
                    window.open(url, '_blank');
                }
            }
        });
    </script>
    """

    final_html = f"<div>{plot_html}</div>{js_script}"

    print("✅ 圖表已生成，點擊圖中資料點可開啟對應超連結。")
    display(HTML(final_html))


except Exception as e:
    print(f"❌ 處理過程中發生錯誤: {e}")

✅ Google Colab 授權成功。
✅ 成功讀取 Google Sheet 資料。

--- 數據分析 (基準: 中位數) ---
中位數觸及人數: 1,294
中位數互動率: 2.8%
✅ 資料清理與準備完成。


/tmp/ipython-input-1161612543.py:50: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['topic_tag'].fillna('未分類', inplace=True)
/tmp/ipython-input-1161612543.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using

✅ 圖表已生成，點擊圖中資料點可開啟對應超連結。
